In [1]:
import math
import numpy as np
from scipy import signal
from PIL import Image, ImageDraw
from scipy.ndimage import gaussian_filter
import os

### Part 1

Question 1-2

In [2]:
import ncc

In [3]:

def MakeGaussianPyramind(image, scale, minsize):
    '''
    purpose: creates a pyramid for an image
    inputs: image - jpg, scale - factor by which to reduce image size, minsize - min dimension of reduced image
    outputs: list of [original_image, PIL images of reduced size]
    '''
    # generate appropriate sigma
    sigma=1/(2*scale)
    # open image
    im = Image.open(image)
    pyr = []
    # set while loop with min dim as stop condition
    x, y = im.height, im.width
    min_dim = np.max([x, y])
    while int(min_dim*0.75) > minsize: # check condition
        if len(im.size) < 3: # if grayscale, do not split channels for separate filtering
            im2 = Image.fromarray(gaussian_filter(im, sigma))
            # resize image
            imnew = im2.resize((int(x*scale), int(y*scale)), Image.BICUBIC)
            # convert back to np array
            resized = np.float32(imnew)
        else:
            r, g, b = im.split() # if multi-channel, split channels and filter before reassmbeling
            r, g, b = Image.fromarray(gaussian_filter(r, sigma)), Image.fromarray(gaussian_filter(g, sigma)), Image.fromarray(gaussian_filter(b, sigma))
            im2 = Image.merge(r,g,b)
            # resize image
            imnew = im2.resize((int(x*scale), int(y*scale)), Image.BICUBIC)
            # convert back to numpy arrays 
            r, b, g = imnew.split()
            resized = np.array([np.float32(r), np.float32(b), np.float32(g)])
        pyr.append(resized)
        # update min_dim
        x, y = imnew.height, imnew.width
        min_dim = np.max([x, y])
    return pyr
        

In [5]:
print(os.getcwd())
filepath=r"hw2part2\faces\judybats.jpg"
pyr = MakeGaussianPyramind(image=filepath, scale=0.75, minsize=100)
print('done')

c:\Users\Sadie\Documents\GitHub\CPSC_425\A2
done


Question 3

In [8]:
def ShowGaussianPyramid(pyr):
    '''
    purpose: display Gaussian pyramid
    inputs: list of np arrays
    outputs: None (display all images in pyramid side by side)
    '''
    heights = list(map(lambda x: x.shape[0], pyr)) # get dims of all imgs in list to know dims needed for template image
    widths = list(map(lambda x: x.shape[1], pyr))
    w_tot, h_tot = np.sum(widths), np.sum(heights)
    image = Image.new(mode="L", size=(w_tot, h_tot), color=1)
    offset_x, offset_y = 0, 0
    for im in pyr:
        image.paste(im,(offset_x, offset_y))
        offset_x, offset_y = im.height, im.width
    display(image)

In [7]:
ShowGaussianPyramid(pyr)

AttributeError: 'numpy.ndarray' object has no attribute 'height'

Question 4

In [ ]:
def FindTemplate(pyramid, template, threshold):
    '''
    purpose: 
    inputs: 
    outputs: 
    '''
    for im in pyramid:
        res = ncc.normxcorr2D(im, template)
        if res > threshold:
            